# 🎵 Music Store Analysis (Chinook Database)

This notebook analyzes the Chinook music store database to uncover revenue drivers, top artists, genres, and customer behavior using **SQL + Python (pandas + matplotlib)**.

In [ ]:
import sqlite3
import pandas as pd
import matplotlib.pyplot as plt

# Connect to Chinook SQLite database (make sure Chinook_Sqlite.sqlite is in the same folder)
conn = sqlite3.connect('Chinook_Sqlite.sqlite')

## 1. Top 10 Artists by Revenue

In [ ]:
query = '''
SELECT ar.Name AS artist_name,
       ROUND(SUM(il.UnitPrice * il.Quantity), 2) AS total_revenue
FROM InvoiceLine il
JOIN Track t ON il.TrackId = t.TrackId
JOIN Album al ON t.AlbumId = al.AlbumId
JOIN Artist ar ON al.ArtistId = ar.ArtistId
GROUP BY ar.Name
ORDER BY total_revenue DESC
LIMIT 10;
'''
df_artists = pd.read_sql(query, conn)
df_artists

In [ ]:
plt.figure(figsize=(9,6))
plt.barh(df_artists['artist_name'][::-1], df_artists['total_revenue'][::-1], color='purple')
plt.title('Top 10 Artists by Revenue')
plt.xlabel('Revenue')
plt.ylabel('Artist')
plt.tight_layout()
plt.show()

## 2. Revenue by Genre

In [ ]:
query = '''
SELECT g.Name AS genre,
       ROUND(SUM(il.UnitPrice * il.Quantity), 2) AS total_revenue
FROM InvoiceLine il
JOIN Track t ON il.TrackId = t.TrackId
JOIN Genre g ON t.GenreId = g.GenreId
GROUP BY g.Name
ORDER BY total_revenue DESC;
'''
df_genres = pd.read_sql(query, conn)
df_genres

In [ ]:
plt.figure(figsize=(10,6))
plt.bar(df_genres['genre'], df_genres['total_revenue'], color='teal')
plt.title('Revenue by Genre')
plt.xlabel('Genre')
plt.ylabel('Revenue')
plt.xticks(rotation=45)
plt.tight_layout()
plt.show()

## 3. Customer Spend Analysis (Lifetime Value)

In [ ]:
query = '''
SELECT c.CustomerId, c.FirstName || ' ' || c.LastName AS customer_name,
       ROUND(AVG(i.Total), 2) AS avg_invoice_amount,
       ROUND(SUM(i.Total), 2) AS lifetime_value
FROM Customer c
JOIN Invoice i ON c.CustomerId = i.CustomerId
GROUP BY c.CustomerId, customer_name
ORDER BY lifetime_value DESC
LIMIT 10;
'''
df_customers = pd.read_sql(query, conn)
df_customers

In [ ]:
plt.figure(figsize=(9,6))
plt.barh(df_customers['customer_name'][::-1], df_customers['lifetime_value'][::-1], color='orange')
plt.title('Top 10 Customers by Lifetime Value')
plt.xlabel('Lifetime Value')
plt.ylabel('Customer')
plt.tight_layout()
plt.show()

## 4. Top Countries by Revenue

In [ ]:
query = '''
SELECT c.Country,
       ROUND(SUM(i.Total), 2) AS total_revenue,
       COUNT(DISTINCT i.InvoiceId) AS num_invoices
FROM Invoice i
JOIN Customer c ON i.CustomerId = c.CustomerId
GROUP BY c.Country
ORDER BY total_revenue DESC
LIMIT 10;
'''
df_countries = pd.read_sql(query, conn)
df_countries

In [ ]:
plt.figure(figsize=(10,6))
plt.bar(df_countries['Country'], df_countries['total_revenue'], color='blue')
plt.title('Top 10 Countries by Revenue')
plt.xlabel('Country')
plt.ylabel('Revenue')
plt.xticks(rotation=45)
plt.tight_layout()
plt.show()

## 5. Most Popular Tracks by Purchase Count

In [ ]:
query = '''
SELECT t.Name AS track_name, ar.Name AS artist_name,
       COUNT(il.InvoiceLineId) AS times_purchased
FROM InvoiceLine il
JOIN Track t ON il.TrackId = t.TrackId
JOIN Album al ON t.AlbumId = al.AlbumId
JOIN Artist ar ON al.ArtistId = ar.ArtistId
GROUP BY t.Name, ar.Name
ORDER BY times_purchased DESC
LIMIT 10;
'''
df_tracks = pd.read_sql(query, conn)
df_tracks

In [ ]:
plt.figure(figsize=(9,6))
plt.barh(df_tracks['track_name'][::-1], df_tracks['times_purchased'][::-1], color='green')
plt.title('Top 10 Tracks by Purchase Count')
plt.xlabel('Times Purchased')
plt.ylabel('Track')
plt.tight_layout()
plt.show()

## 6. Monthly Revenue Trend with Cumulative Revenue

In [ ]:
query = '''
SELECT strftime('%Y-%m', i.InvoiceDate) AS month,
       ROUND(SUM(i.Total), 2) AS monthly_revenue
FROM Invoice i
GROUP BY month
ORDER BY month;
'''
df_monthly = pd.read_sql(query, conn)
df_monthly['cumulative_revenue'] = df_monthly['monthly_revenue'].cumsum()
df_monthly

In [ ]:
plt.figure(figsize=(10,6))
plt.plot(df_monthly['month'], df_monthly['monthly_revenue'], marker='o', label='Monthly Revenue')
plt.plot(df_monthly['month'], df_monthly['cumulative_revenue'], marker='x', label='Cumulative Revenue')
plt.title('Monthly and Cumulative Revenue Trend')
plt.xlabel('Month')
plt.ylabel('Revenue')
plt.xticks(rotation=45)
plt.legend()
plt.tight_layout()
plt.show()